# Error Analysis (2)

From the previous notebook, you have seen that pyConTextNLP helped to improve the precision by excluding irrelevant annotations based on the modifiers. 

This notebook will continue our previous error analyses by including both false negative errors and false positive ones, through step by step demonstration of how to locate and reduce the errors.

## 1. Locate the errors

In [281]:
# Let's import some packages
import os
import pyConTextNLP
from pyConTextNLP import pyConText
import sklearn.metrics
import pandas as pd
import networkx as nx
import radnlp.view as rview

from ipywidgets import interact, interactive, fixed, interact_manual
from IPython.display import display, HTML, Image
import ipywidgets
# And also our utilities for this class

from nlp_pneumonia_utils import read_doc_annotations
from nlp_pneumonia_utils import mark_document_with_html
from DocumentClassifier import DocumentClassifier
from visual import view_pycontext_output
from visual import snippets_markup

Remember yesterday, in [06_NLP_ErrorAnalysis1](06_NLP_ErrorAnalysis1.ipynb#cell1), we created a function called *"list_false_negatives."* Now we will extend this function to return both **false negative** and **false positive** document names at the same time: *"list_errors."* Additionally, we will also integrate the *"calculate_prediction_metrics"* function inside *"list_errors"*, so that we can get the metrics without re-run the pyConText over the documents again.
<br/><br/>


In [357]:
def list_errors(gold_docs, prediction_function,
                print_prediction_metrics=False):
    fn_docs = []
    fp_docs = []
    gold_labels = [x.positive_label for x in gold_docs.values()]
    pred_labels = []
    for doc_name, gold_doc in gold_docs.items():
        gold_label = gold_doc.positive_label;
        pred_label = prediction_function(gold_doc.text, doc_name)
        pred_labels.append(pred_label)
        #       Differentiate false positive and false negative error
        if gold_label == 0 and pred_label == 1:
            fp_docs.append(doc_name)
        elif gold_label == 1 and pred_label == 0:
            fn_docs.append(doc_name)
    if (print_prediction_metrics):
        precision = sklearn.metrics.precision_score(gold_labels, pred_labels)
        recall = sklearn.metrics.recall_score(gold_labels, pred_labels)
        f1 = sklearn.metrics.f1_score(gold_labels, pred_labels)
        # Let's use Pandas to make a confusion matrix for us
        se1 = pd.Series(gold_labels, name='Actual');
        se2 = pd.Series(pred_labels, name='Predicted')
        confusion_matrix_df = pd.crosstab(pd.Series(gold_labels, name='Actual'),
                                          pd.Series(pred_labels, name='Predicted'))

        print('Precision : {0}'.format(precision))
        print('Recall :    {0}'.format(recall))
        print('F1:         {0}'.format(f1))

        print('\nConfusion Matrix : ')
        print(confusion_matrix_df)
    return fn_docs, fp_docs  


Now we restore what we got from [10_NLP_DocumentClassification.ipynb](10_NLP_DocumentClassification.ipynb):<br/><br/>

In [358]:
#?DocumentClassifier

In [395]:
#Read in the training documents and annotations
annotated_doc_map = read_doc_annotations('data/test_v2.zip')
#annotated_doc_map = read_doc_annotations('data/training_v2.zip')


#Here we initiate our DocumentClassifier directly through rule files:
#Change the file names if you use different files 
pos_doc_type='PNEUMONIA_DOC_YES'
TARGETS_FILE_PATH = 'KB/pneumonia_targets.yml'
MODIFIERS_FILE_PATH = 'KB/pneumonia_modifiers.yml'
FEATURE_INFERENCER_FILE_PATH = 'KB/featurer_inferences.csv'
DOC_INFERENCER_FILE_PATH = 'KB/doc_inferences.csv'
# clear just in case files/regular expressions have been updated
classifier = DocumentClassifier(TARGETS_FILE_PATH, MODIFIERS_FILE_PATH,
                               FEATURE_INFERENCER_FILE_PATH, DOC_INFERENCER_FILE_PATH,
                               {pos_doc_type})

Reading annotations from file : data/test_v2.zip
Opening local file : data/test_v2.zip


In [396]:
# Process the corpus using docClassifier to return errors
current_false_negatives,current_false_positives=list_errors(annotated_doc_map, classifier.predict,True)

Precision : 0.8125
Recall :    0.9285714285714286
F1:         0.8666666666666666

Confusion Matrix : 
Predicted   0   1
Actual           
0          13   3
1           1  13


DocumentClassifier also has a wrapped-up function to do this evaluation:

In [397]:
current_false_negatives, current_false_positives, measurements,confusion_matrix_df = classifier.eval(annotated_doc_map)
print(measurements)
display(confusion_matrix_df)

Start to evaluate against reference standards...
Precision : 0.812
Recall :    0.929
F1:         0.867


Predicted,1,0
Actual,,
1,13,1
0,3,13


## 2. Display errors
Now we put everything together to display errors. Let's try false negative first:<br/><br/>

### * Display false negatives
Now we can display the **false negatives** with expert annotations.<br/><br/>

In [398]:
fn_docs=dict((k, v) for k, v in annotated_doc_map.items() if k in current_false_negatives)
display(HTML(snippets_markup(fn_docs)))

document name,Snippets
subject_id_4276_hadm_id_25705,
,"the carina. IMPRESSION: 1. New bibasilar opacities, which may represent atelectasis or aspiration pneumonia. 2. Right central venous catheter with"
,"FINDINGS: Since prior examination, has been interval development of bibasilar opacities, may represent atelectasis or aspiration pneumonia. Right- sided subclavian approach central ve"


### * Display false positives
Then we can display the **false positives** with pyConText markups.<br/><br/>

In [399]:
fp_docs = dict((k,v) for k, v in classifier.saved_markups_map.items() if k in current_false_positives)
view_pycontext_output(fp_docs)

interactive(children=(IntSlider(value=0, description='i', max=2), Output()), _dom_classes=('widget-interact',)…

### * Rethink the causes of the false negatives
Does the false negatives are all caused by missed keywords?<br/><br/>
You may want to review these pyConText markups in **false negative** documents:<br/><br/>


In [389]:
fn_docs = dict((k,v) for k, v in classifier.saved_markups_map.items() if k in current_false_negatives)
view_pycontext_output(fn_docs)
# The variable: "saved_markups_map" in "docClassifier" only saves the documents that have at least one annotation. 
# Thus, the false negatives caused by missing keywords (no annotations) will not be saved in it,
# and only pyConText caused false negatives will be displayed here

interactive(children=(IntSlider(value=0, description='i', max=0), Output()), _dom_classes=('widget-interact',)…

## 3. Now what?<br/><br/>

## 4. Quiz
Let's see if you are ready.

In [9]:
from quiz_utils import error_analyses_7
error_analyses_7()

RadioButtons(description='Choose the best answer:', layout=Layout(width='600px'), options=('A', 'B', 'C', 'D')…

Button(description='Submit', style=ButtonStyle())

In [10]:
from quiz_utils import error_analyses_8
error_analyses_8()

RadioButtons(description='Choose the best answer:', layout=Layout(width='600px'), options=('A', 'B', 'C', 'D')…

Button(description='Submit', style=ButtonStyle())

In [11]:
from quiz_utils import error_analyses_9
error_analyses_9()

RadioButtons(description='Choose the best answer:', layout=Layout(width='600px'), options=('A', 'B', 'C', 'D')…

Button(description='Submit', style=ButtonStyle())

<br/><br/>This material presented as part of the DeCART Data Science for the Health Science Summer Program at the University of Utah in 2019.<br/>
Presenters : Dr. Wendy Chapman, Kelly Peterson, Alec Chapman, Jianlin Shi <br> Acknowledgement: Many thanks to Olga Patterson because part of the materials are adopted from his previous work.